In [2]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataset import random_split

import dgl
from dgl import function as fn
from dgl import DGLGraph
from dgl.data import citation_graph as citegrh

import networkx as nx
import matplotlib.pyplot as plt
import random as rand
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [3]:


#set gpu if available
if th.cuda.is_available():
    print("GPU is available")
    #device = th.device("cuda")
    device = th.device("cuda")
else:
    print("GPU not available, CPU used")
    device = th.device("cpu")

GPU is available


In [4]:
#operation for neigbors
class NodeApplyModule(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation

    def forward(self, node):
        h = self.linear(node.data['h'])
        if self.activation is not None:
            h = self.activation(h)
        return {'h' : h}
    
#gcn layer in network
class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        g.ndata['h'] = feature
        g.pull(g.nodes())
        g.apply_nodes(self.apply_mod)
        
        return g.ndata.pop('h')
    
#network
class LIGN(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(LIGN, self).__init__()
        self.gcn1 = GCN(in_feats, 100, F.relu)
        self.gcn2 = GCN(100, 30, F.relu)
        self.gcn3 = GCN(30, out_feats, th.tanh)

    def forward(self, g, features):
        x = self.gcn1(g, features)
        x = self.gcn2(g, x)
        
        return self.gcn3(g, x)


In [5]:
#loss function

def similarity_matrix(x):
    x_norm = (x**2).sum(1).view(-1, 1)
    y = x
    y_norm = x_norm.view(1, -1)

    dist = x_norm + y_norm - 2.0 * th.mm(x, th.transpose(y, 0, 1))
    return dist

def same_label(y):
    s = y.size(0)
    y_expand = y.unsqueeze(0).expand(s, s)
    Y = y_expand.eq(y_expand.t())
    return Y

def my_loss(output, labels):
    """
    if nodes with the same label: x^2
    if nodes with different label: 1/(10*x^2)
    """
    sim = similarity_matrix(output)
    temp = same_label(labels)
    same_l = (temp * sim) / (output.size(0)**2)
    same_l_inv = ((temp*(-1) + 1) * sim) / (output.size(0)**2)
    
    loss = ((th.sum(same_l)**2) + (1/(10*th.sum(same_l_inv)**2)))
    
    return loss

"""def has_2_rand_label(nodes): #return nodes of two unseen labels
    rand.shuffle(possible_lab)
    
    out = (nodes.data['t_label'] == possible_lab[0]).squeeze(1)
    out.extend((nodes.data['t_label'] == possible_lab[1]).squeeze(1))
    
    used_lab.append(possible_lab.pop(0).item())
    used_lab.append(possible_lab.pop(1).item())
    
    return out"""

def has_1_rand_label(nodes): return (nodes.data['t_labels'] == possible_lab[0]).squeeze(0)

def filter_knn_label(nodes): return (nodes.data['t_labels'] == curr_used_lab).squeeze(0)



In [6]:
#load dataset
data = citegrh.load_cora()
ds_features = th.FloatTensor(data.features).to(device) #convert to pytorch data type and add to cpu/gpu
ds_labels = th.LongTensor(data.labels).to(device)
ds_g = data.graph

# add self loop for the sum of festures
ds_g.remove_edges_from(nx.selfloop_edges(ds_g))
ds_g = DGLGraph(ds_g)
ds_g.add_edges(ds_g.nodes(), ds_g.nodes())
ds_g.ndata['features'] = ds_features
ds_g.ndata['t_labels'] = ds_labels #used to filter and train the first two labels, not needed for prediction

# to coordinate sending of features over the graph network
m_func = fn.copy_src(src='h', out='m')
m_reduce_func = fn.sum(msg='m', out='h')

In [7]:
########### Create Model ############

#constant parameters
DIST_VEC_SIZE = 10
NUMBER_OF_LABELS = 7

model = LIGN(ds_features.size()[1], DIST_VEC_SIZE).to(device)
knn = KNeighborsClassifier(n_neighbors=3)
opt = th.optim.Adam(model.parameters(), lr=1e-3)# only run once

In [ ]:
#training

train_g = ds_g.subgraph(ds_g.nodes()[:int(len(ds_g) * .80)])#80 percent of all the nodes
test_g = ds_g.subgraph(ds_g.nodes()[int(len(ds_g) * .80):]) #20 percent labeled nodes for knn
train_g.copy_from_parent()
test_g.copy_from_parent()

possible_lab = list(range(NUMBER_OF_LABELS))
rand.shuffle(possible_lab) #set random order for how labels are added
used_lab = []
curr_used_lab = 0

EPOCH = 50

model.train()

#############train with 2 labels of 7 labels (uses true label from the dataset), assign vector to each node,
            #knn (1 labeled node for each label), test
selected_nodes = train_g.filter_nodes(has_1_rand_label) #add two unseen labels
used_lab.append(possible_lab.pop(0))
selected_nodes = th.cat((train_g.filter_nodes(has_1_rand_label), selected_nodes), -1)
used_lab.append(possible_lab.pop(0))

c=th.randperm(len(selected_nodes)) #shuffle
selected_nodes=selected_nodes[c]

#train
for epoch in range(EPOCH):
    print("> " + str(epoch))
    c=th.randperm(len(selected_nodes)) #shuffle
    selected_nodes=selected_nodes[c]
    epoch_nodes = selected_nodes[:int(len(selected_nodes)*.20)] #selected 20% of random nodes to train with at each epoch
    
    error = []
    for count in range(len(epoch_nodes)):
        sub_graph = train_g.subgraph(epoch_nodes[:count])
        sub_graph.copy_from_parent()
        sub_graph.register_message_func(m_func)
        sub_graph.register_reduce_func(m_reduce_func)
        
        feats = sub_graph.ndata['features']
        labs = sub_graph.ndata['t_labels'] #true label
        
        out = model(sub_graph, feats)
        loss = my_loss(out, labs)
        error.append(loss.item())
        
        opt.zero_grad()
        loss.backward()
        opt.step()
    
#knn and assign vector
sub_tra_graph = train_g.subgraph(selected_nodes)
sub_tra_graph.copy_from_parent()
sub_tra_graph.register_message_func(m_func)
sub_tra_graph.register_reduce_func(m_reduce_func)

##test graph knn
out_nodes = th.FloatTensor([])
for n in used_lab:
    curr_used_lab = n
    out_nodes = th.cat((test_g.filter_nodes(has_1_rand_label), out_nodes), -1)

c=th.randperm(len(out_nodes)) #shuffle
out_nodes=out_nodes[c]

sub_tes_graph = test_g.subgraph(out_nodes)
sub_tes_graph.copy_from_parent()
sub_tes_graph.register_message_func(m_func)
sub_tes_graph.register_reduce_func(m_reduce_func)

knn.fit(model(sub_tes_graph, sub_tes_graph.ndata['features']), sub_tes_graph.ndata['t_labels'])
sub_tra_graph.ndata['p_labels'] = knn.predict(model(sub_tra_graph, sub_tra_graph.ndata['features']))

#test
print(f"Accuracy: {metrics.accuracy_score(sub_tra_graph.ndata['t_labels'], sub_tra_graph.ndata['p_labels'])}")
    

> 0
> 1


In [7]:
  
#training 
#######################add label, assign vector to each node, knn, test and train (with predicted labels by knn)

while len(possible_lab) > 0: #do it for every unseen label
    selected_nodes = th.cat((train_g.filter_nodes(has_1_rand_label), selected_nodes), -1) #add unseen new label (change 1 to 2 to add to labels at once)
    used_lab.append(possible_lab.pop(0))
    c=th.randperm(len(selected_nodes)) #shuffle
    selected_nodes=selected_nodes[c]
    
    #assign vector
    #knn
    #test
    #train
    for epoch in range(EPOCH):
        c=th.randperm(len(selected_nodes)) #shuffle
        selected_nodes=selected_nodes[c]
        epoch_nodes = selected_nodes[:int(len(selected_nodes)*.20)] #select 20% of random nodes to train with at each epoch

        error = []
        for count in range(len(epoch_nodes)):
            sub_graph = train_g.subgraph(epoch_nodes[:count])
            sub_graph.copy_from_parent()
            sub_graph.register_message_func(m_func)
            sub_graph.register_reduce_func(m_reduce_func)

            feats = sub_graph.ndata['features']
            
            labs = sub_graph.ndata['t_labels'] #change to predicted label by knn (if label not given) *********

            out = model(sub_graph, feats)
            loss = my_loss(out, labs)
            error.append(loss.item())

            opt.zero_grad()
            loss.backward()
            opt.step()

Epoch 0: 
	avg error = 2.849159374212042e-06 
	last error = 3.274331472624681e-09 
	avg accuracy = 0.0 
	last accuracy = 0.0 
Epoch 1: 
	avg error = 8.396367201980384e-09 
	last error = 1.2516924163818999e-09 
	avg accuracy = 0.0 
	last accuracy = 0.0 
Epoch 2: 
	avg error = 3.1999168450808323e-09 
	last error = 9.623448704587645e-10 
	avg accuracy = 0.0 
	last accuracy = 0.0 
Epoch 3: 
	avg error = 1.4447087481560294e-09 
	last error = 3.491356093121567e-10 
	avg accuracy = 0.0 
	last accuracy = 0.0 
Epoch 4: 
	avg error = 7.560877813046131e-10 
	last error = 1.263662063877291e-09 
	avg accuracy = 0.0 
	last accuracy = 0.0 
Epoch 5: 
	avg error = 4.4562802434009954e-10 
	last error = 7.072740171309988e-10 
	avg accuracy = 0.0 
	last accuracy = 0.0 
Epoch 6: 
	avg error = 2.901853388827861e-10 
	last error = 5.298499139438917e-11 
	avg accuracy = 0.0 
	last accuracy = 0.0 
Epoch 7: 
	avg error = 2.0170063083032827e-10 
	last error = 2.8992380718406707e-10 
	avg accuracy = 0.0 
	last ac

In [9]:
ds_g.ndata['t_label'][:10]

tensor([2, 5, 4, 4, 3, 3, 6, 2, 2, 6])

In [11]:
list = []
list.append(1)